# Taller Fourier EDM - Clasificación de cargas de electrodomésticos

## Práctica 2 - Adquisición y análisis de señales con la tarjeta Analog Discovery 2

### Objetivos:
En esta práctica se busca familiarizarse con la tarjeta Analog Discovery 2 y realizar la adquisición de señales mediante el software WaveForms y programas en python.

### Actividades
1. Familiarizarse con el software WaveForms
1. Usar y programar funciones de adquisición para la tarjeta Analog Discovery 2
1. Adquirir, guardar y graficar señales


### Materiales

Software WaveForms y SDK  
https://reference.digilentinc.com/reference/software/waveforms/waveforms-3/start    
https://reference.digilentinc.com/_media/waveforms_sdk_reference_manual.pdf




### Trabajo a realizar

A lo largo del notebook encontrarán las siguientes palabras claves:

* \# COMPLETAR  -> indica que se requiere completar el código. 
  
  
* \# EXPERIMENTAR -> realizar experimentos y mostrar resultados
  
  
* \# DISCUTIR -> se espera una discusión de los experimentos realizados y/o de las preguntas formuladas
  
  
* \# RESPONDER -> se espera una respuesta a preguntas concretas

In [22]:
import os
import sys
import numpy as np
from scipy.fftpack import fft, fftshift, ifft
from scipy.signal import spectrogram
import matplotlib.pyplot as plt

#estilo de las gráficas
plt.style.use('ggplot')


# graficas en línea  entre las celdas
#%matplotlib inline 
# graficas en línea  entre las celdas con pan/zoom
%matplotlib notebook
# graficas en ventanas externas
# %matplotlib



In [51]:
###Funciones de carga de datos de la Parte 1

def cargar_VI_por_ciclos(nombre_archivo, 
                         frecuencia_muestreo=30000,
                         frecuencia_linea=60,
                         ciclos_a_cargar=1e100,  #por defecto se cargan todos los ciclos posibles
                         ciclos_a_saltear=0):    #por defecto se carga desde el inicio
    '''
    Carga un cierto numero de ciclos de una señal I,V guardada en un archivo
    Devuelve las señales I y V como  2 arrays Nx1
    
    Importante: se debe asegurar que se carga un número entero de ciclos (el número final
    de ciclos cargados podría eventualmente ser menor a 'ciclos_a_cargar')
    '''
    
    # COMPLETADO
    I = np.genfromtxt(nombre_archivo, delimiter=',', skip_header=1)[:, 0:1].flatten()
    V = np.genfromtxt(nombre_archivo, delimiter=',', skip_header=1)[:, 1:2].flatten()
    
    muestras_por_ciclo = int(frecuencia_muestreo/frecuencia_linea)
    cantidad_ciclos_completos = int(len(I)/muestras_por_ciclo)
    
    muestra_inicial = int(ciclos_a_saltear*muestras_por_ciclo)
    muestra_final = min(muestra_inicial + int(ciclos_a_cargar*muestras_por_ciclo), cantidad_ciclos_completos*muestras_por_ciclo)
            
    I = I[muestra_inicial:muestra_final]
    V = V[muestra_inicial:muestra_final]    
    
    return I,V


def generar_vector_tiempo(numero_muestras, frecuencia_muestreo=30000):
    '''
    Genera un vector de tiempo del largo solicitado
    '''
    duracion = (numero_muestras-1)/frecuencia_muestreo   #duración de la señal en segundos
    vector_tiempo = np.linspace(0,duracion,numero_muestras)
    
    return vector_tiempo


def graficar_I_V(I, V, frecuencia_muestreo=30000, fignum=None, color = 'g'):
    '''
    Genera un vector de tiempos T adecuado y grafica por separado
    las señales de corriente I(T) y voltaje V(T) que se le pasan.
    Se supone que I y V son de igual largo
    
    Si se le pasa un fignum, grafica I sobre la figura (fignum) y V sobre la 
    figura (fignum+1). De lo contrario crea dos figuras nuevas.
    '''
    
    T = generar_vector_tiempo(len(I), frecuencia_muestreo)
    
    # COMPLETADO
    
    #Grafica corrientes
    plt.figure(fignum)
    plt.plot(T, I, color)
    plt.title("$I = f(t)$")
    plt.xlabel("Tiempo $t$")
    plt.ylabel("Corriente $I$")
    
    #Grafica voltajes
    if(fignum == None):
        plt.figure()
    else:
        plt.figure(fignum + 1)
    plt.plot(T, V, color)
    plt.title("$V = f(t)$")
    plt.xlabel("Tiempo $t$")
    plt.ylabel("Voltaje $V$")
    
def graficar_diagrama_VI(I, V, fignum=None):
    '''
    Grafica I vs. V 
    
    Si se le pasa un fignum, grafica el diagrama sobre la 
    figura (fignum). De lo contrario crea una figuras nueva.
    '''
    if not fignum is None:
        plt.figure(fignum)
    else:
        plt.figure()
        
    plt.plot(V,I,'.-')
    plt.title('Diagrama V-I')
    plt.xlabel('V')
    plt.ylabel('I')

###Funciones de análisis en frecuencia de la Parte 1

def generar_vector_frecuencia(numero_muestras, frecuencia_muestreo=30000, centrar_frecuencia=True):
    '''
    Genera un vector de frecuencias del largo especificado
    If centrar_frecuencia==True (por defecto)
        salida es un array   [-Fm/2.....Fm/2-Fm/numero_muestras]
    else
        salida es un array   [0.....Fm-Fm/numero_muestras]
    '''
    step_frecuencia = frecuencia_muestreo/numero_muestras 
    vector_frecuencia = np.arange(0,frecuencia_muestreo,step_frecuencia)
    if centrar_frecuencia:
        vector_frecuencia = vector_frecuencia - frecuencia_muestreo/2
    
    return vector_frecuencia


def graficar_FI_FV(I,V, frecuencia_muestreo=30000, centrar_frecuencia=True, fignum=None):
    '''
    Genera un vector de frecuencia adecuado.
    Grafica el modulo de la transformada de I y de V en figuras separadas
    
    Si se le pasa un fignum, grafica FI sobre la figura (fignum) y FV sobre la 
    figura (fignum+1). De lo contrario crea dos figuras nuevas.
    '''
    numero_muestras = len(I)
    vector_frecuencia = generar_vector_frecuencia(numero_muestras, frecuencia_muestreo, centrar_frecuencia)
    
    # COMPLETADO
    
    
    #Grafica corrientes
    plt.figure(fignum)
    if centrar_frecuencia:
        plt.plot(vector_frecuencia, np.abs(fftshift(fft(I))), 'r')
    else:
        plt.plot(vector_frecuencia, np.abs(fft(I)), 'r')
        
    plt.title("$abs(fft(I)) = f(f)$")
    plt.xlabel("Frecuencia $f$")
    plt.ylabel("$fft(I)$")
    
    #Grafica voltajes
    if(fignum == None):
        plt.figure()
    else:
        plt.figure(fignum + 1)
        
    if centrar_frecuencia:
        plt.plot(vector_frecuencia, np.abs(fftshift(fft(V))), 'r')
    else:
        plt.plot(vector_frecuencia, np.abs(fft(V)), 'r')
        
    plt.title("$abs(fft(V)) = f(f)$")
    plt.xlabel("Frecuencia $f$")
    plt.ylabel("$fft(V)$")
    
    

def graficar_espectrograma_I_V(I,V, frecuencia_muestreo=30000, largo_ventana=256, fignum=None):
    '''
    Grafica el espectrograma de I y de V en figuras separadas
    
    Si se le pasa un fignum, grafica el espectrograma de I sobre la figura (fignum)
    y el de V sobre la figura (fignum+1). De lo contrario crea dos figuras nuevas.
    '''
    f,t,SI = spectrogram(I,fs=frecuencia_muestreo, nperseg=largo_ventana)
    f,t,SV = spectrogram(V,fs=frecuencia_muestreo, nperseg=largo_ventana)
    
    
    if not fignum is None:
        plt.figure(fignum)
    else:
        plt.figure()
    
    plt.pcolormesh(t, f, SI)
    plt.title('Espectrograma de I')
    plt.ylabel('Frecuencia [Hz]')
    plt.xlabel('Tiempo [s]')
    
    if not fignum is None:
        plt.figure(fignum)
    else:
        plt.figure()
    
    plt.pcolormesh(t, f, SV)
    plt.title('Espectrograma de V')
    plt.ylabel('Frecuencia [Hz]')
    plt.xlabel('Tiempo [s]')

In [3]:
### Para la parte 3
from ctypes import *
from funciones.dwfconstants import *
import time

## Parte 1 - Uso del software WaveForms  
Esta parte de la práctica no requiere la realización de código en este notebook.   
Realizar las pruebas con la tarjeta AD2 y el software WaveForms. Comentar y agregar imágenes en el notebook o en  otro documento de los experimentos realizados.

### 1.1 Conectar generador y osciloscopio de la tarjeta AD2 y analizar señales

#### 1.1.1 Analizar señales del generador (sinusoide, onda cuadrada, onda triangular, rampa)


# EXPERIMENTAR   
Generar señales de 50 Hz y analizar con FFT y Espectrograma.

In [1]:
###Además de lo pedido, probamos analizar con las funciones que creamos en la Práctica 1

## Experimentamos con las funciones de la Práctica 1

#T, V = cargar_VI_por_ciclos('./relevadas/OscilosopioSin50Hz4000SR.csv', frecuencia_muestreo=4000, frecuencia_linea=50)
#graficar_FI_FV(T, V, frecuencia_muestreo=4000)

#T, V = cargar_VI_por_ciclos('./relevadas/OscilosopioSin50Hz.csv', frecuencia_muestreo=400000, frecuencia_linea=50)
#graficar_FI_FV(T, V, frecuencia_muestreo=400000)

#graficar_diagrama_VI(V,T)

#print(T,V)

## Experimentamos con las herramientas del GUI Waveforms

# EXPERIMENTAR y RESPONDER    

### Para la FFT:

* Qué separación hay entre muestras en frecuencia? Justifique.
##### La separación entre muestras es la frecuencia de muestreo divida la cantidad de muestras. Exportando datos desde el WaveForms siempre sacamos una cantidad fija de muestras, pero dependiendo la escala del eje horizontal podemos variar la frecuencia de muestreo. En los ejemplos de arriba, tenemos separación entre muestras de 0,5 y 50 Hz respectivamente.


* Para la señal sinusoidal, ¿por qué no se ve la FFT exactamente como una delta?
##### Por un lado, esto es porque las señales analizadas tienen ruido de altas frecuencias que se mezclan con la potencia de la señal. Por otro lado, al enventanar en el tiempo, se deforma el espectro de la señal en frecuencia, nunca podremos ver exactamente una delta, dado que precisaríamos infinitas muestras en el tiempo.


* Para la señal sinusoidal, experimente con distintas ventanas (rectangular, hamming, hann, flat-top). ¿Cómo afecta la ventana elegida a la FFT?
##### Como no se puede analizar la señal en tiempo infinito, se recorta una parte de la señal en el tiempo con una "ventana". Como la señal en el tiempo queda el producto de la señal original con la ventana, en frecuencia tenemos una convolución de sus FFT. Por ende, el "tipo" de ventana que se use definirá la FFT de la misma y como se deforma el espectro de la señal original. Por ejemplo, con la ventana Hamming vemos más lóbulos, mientras que con la rectangular vemos que la delta decae mas "suavemente". Sin embargo, con la ventana Hamming tenemos mucho más atenuación a altas frecuencias.

![Ventana rectangular](./relevadas/Rectangular.png "Ventana rectangular")
![Ventana flattop](./relevadas/FlatTop.png "Ventana flattop")


* Para las señales onda cuadrada y onda triangular, en qué caso decaen los componentes más rápido al aumentar la frecuencia? Por qué ?
##### Observamos que la ventana que más rápido hace decaer a los espectros en ambos casos es la de Blackman-Harris, sin embargo, tiene más lóbulos que otras ventanas en altas frecuencias. Para la señal triangular, notamos el peor desempeño con la ventana rectangular. También notamos que con la señal rectangular, es más dificil lograr un buen enventanado, ya que tiene muchos componentes de alta frecuencia causados por las discontinuidades. 


* Para las señales onda triangular y rampa, qué componentes armónicos observa? ¿Por qué se da esa diferencia?
##### Notamos que la rampa tiene muchos más armónicos de alta frecuencia, lo cual es esperable dada las discontinuidades de esta señal, que se interpretan como armónicos de muy alta frecuencia (es un salto casi instantáneo en el tiempo). 


### Para el Espectrograma:

* Para la onda cuadrada, experimente con distinta resoluciones. De qué manera afecta la resolución al cálculo del espectrograma? Qué observa ?
##### Si la resolución es demasiado chica, no se nota diferenciar en el espectrograma los cambios en el espectro que ocurren a medida que pasa el tiempo. Si en cambio es demasiado grande, la ventana en el tiempo quizás no llega a abarcar un ciclo completo de la señal y por ende el espectrograma no es representativo de la señal. 


#### 1.1.2 Analizar señales de la base PLAID

# EXPERIMENTAR   

Importar señales de distintos electrodomésticos de la base PLAID y analizar

##### En esta parte, importamos señales del PLAID al Waveforms usando Scope>File>Import, seteando Sample rate = 30kHz. 

##### Veamos por ejemplo el electrodoméstico ID 10, que sabemos es un microondas:

![Voltaje ID=10 (Microwave)](./VoltajeID=10(Microwave).png "Voltaje ID=10 (Microwave)")

![Corriente ID=10 (Microwave)](./CorrienteID=10(Microwave).png "Corriente ID=10 (Microwave)")

##### Vemos que el voltajetiene espectro aproximademente constante (esperable dado que es el voltaje de línea, sinusoidal), mientras que el espectro de la corriente varía con el tiempo. Esto coincide con lo visto en la práctica 1, dónde discutimos que estos comportamientos transitorios en por ejemplo el microondas podrían deberse a distintas partes del ciclo de operación del mismo (encendido del motor, etc.).  

##### Comparamos esto con el espectrograma de una lámpara incandecente (ID 57):

![CorrienteID=57(Incandescent Light Bulb)](./CorrienteID=57(IncandescentLightBulb).png "CorrienteID=57(Incandescent Light Bulb)")

##### Vemos que el espectro es casi constante, esperable para un elemento resistivo, cuya corriente es una señal sinusoidal. 

## Parte 2 - Uso de los programas del SDK de WaveForms
Esta parte de la práctica no requiere la realización de código en este notebook.   
Realizar las pruebas con la tarjeta AD2 y los programas de ejemplo del SDK. Comentar y agregar imágenes en el notebook o en  otro documento de los experimentos realizados.

### 2.1 Usar el programa de ejemplo AnalogIn_Record.py para adquirir una sinusoide

#### 2.1.1 Analizar el código. Agregar al código comentarios que indiquen qué se realiza en cada parte.

# EXPERIMENTAR

#####  Adjunto en el zip se encuentra el código con comentarios, en ./funciones/AnalogIn_Record.py

#### 2.1.2 Ejecutar el ejemplo

# EXPERIMENTAR

##### El resultado de la ejecución se ve en la siguiente imágen:

![AnalogIn_Record](./funciones/AnalogIn_Record.jpg "AnalogIn_Record")

##### Vemos que el resultado es el esperado: El generador de ondas emite un seno que el osciloscopio lee, registra y luego grafica.

## Parte 3 - Adaptar el programa de la parte 2 al notebook

### 3.1 Completar la funcion de adquisición de un canal

Partiendo del ejemplo AnalogIn_Record.py del SDK de la tarjeta, crear una función que adquiera la señal conectada al primer canal (CH0) del osciloscopio.   

Comentar adecuadamente las partes del código (similar a 2.1.1)

In [ ]:
from ctypes import *
import math
import time
import numpy as np

##Se agregó un parámetro "canal" para realizar la siguiente parte.
##Usandolo se especifíca que canal de osciloscopio se quiere usar.
## canal=0 <--> ch0 // canal=1 <--> ch1
def adquirir_un_canal(cantidad_de_muestras, frecuencia_de_muestreo = 25000, 
                     rango_canal = 10,
                     canal = 0,
                     generar_sinusoide=False):
    '''
    Adquiere la señal conectada al canal ch0 del osciloscopio.
    
    Opcionalmente genera una sinusoide por el primer canal del generador de señales por 
    si se quiere enviar esta señal al osciloscopio.
    
    Devuelve las muestras adquiridas como un array de numpy.
    '''
    
    # COMPLETAR
    
    ###Se fija en que sistema operativo se está corriendo
    if sys.platform.startswith("win"):
        dwf = cdll.dwf
    elif sys.platform.startswith("darwin"):
        dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
    else:
        dwf = cdll.LoadLibrary("libdwf.so")

    ###Crea variables compatibles con C para poder comunicarse con el AD2
    hdwf = c_int()
    sts = c_byte()
    hzAcq = c_double(frecuencia_de_muestreo)
    nSamples = cantidad_de_muestras
    rgdSamples = (c_double*nSamples)()
    cAvailable = c_int()
    cLost = c_int()
    cCorrupted = c_int()
    fLost = 0
    fCorrupted = 0

    ###Imprime la version de DWF
    version = create_string_buffer(16)
    dwf.FDwfGetVersion(version)
    print("DWF Version: "+str(version.value))

    ###Se conecta con el AD2
    print("Opening first device")
    dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))

    ###Try/Catch de que se haya conectado bien
    if hdwf.value == hdwfNone.value:
        szerr = create_string_buffer(512)
        dwf.FDwfGetLastErrorMsg(szerr)
        print(str(szerr.value))
        print("failed to open device")
        quit()

    ###Se genera un seno
    if(generar_sinusoide):
        print("Generating sine wave...")
        dwf.FDwfAnalogOutNodeEnableSet(hdwf, c_int(0), AnalogOutNodeCarrier, c_bool(True))
        dwf.FDwfAnalogOutNodeFunctionSet(hdwf, c_int(0), AnalogOutNodeCarrier, funcSine)
        dwf.FDwfAnalogOutNodeFrequencySet(hdwf, c_int(0), AnalogOutNodeCarrier, c_double(1))
        dwf.FDwfAnalogOutNodeAmplitudeSet(hdwf, c_int(0), AnalogOutNodeCarrier, c_double(2))
        dwf.FDwfAnalogOutConfigure(hdwf, c_int(0), c_bool(True))

    ###Se configura el canal de adquisición de muestras
    #Habilita el ch0 del osciloscopio (c_int(0))
    dwf.FDwfAnalogInChannelEnableSet(hdwf, c_int(canal), c_bool(True))
    #Setea el rango del canal a +-5V
    dwf.FDwfAnalogInChannelRangeSet(hdwf, c_int(canal), c_double(rango_canal))
    #Configura el ch0 para adquirir en modo: por periodo de tiempo configurado
    dwf.FDwfAnalogInAcquisitionModeSet(hdwf, acqmodeRecord)
    #Setea la frecuencia de muestreo
    dwf.FDwfAnalogInFrequencySet(hdwf, hzAcq)
    #Setea largo de tiempo a adquirir
    dwf.FDwfAnalogInRecordLengthSet(hdwf, c_double(nSamples/hzAcq.value)) # -1 infinite record length

    #wait at least 2 seconds for the offset to stabilize
    time.sleep(2)

    print("Starting oscilloscope")
    dwf.FDwfAnalogInConfigure(hdwf, c_int(0), c_int(1))

    cSamples = 0

    ###Se toman las muestras
    while cSamples < nSamples:
        dwf.FDwfAnalogInStatus(hdwf, c_int(1), byref(sts))
        if cSamples == 0 and (sts == DwfStateConfig or sts == DwfStatePrefill or sts == DwfStateArmed) :
            # Acquisition not yet started.
            continue

        dwf.FDwfAnalogInStatusRecord(hdwf, byref(cAvailable), byref(cLost), byref(cCorrupted))

        cSamples += cLost.value

        if cLost.value :
            fLost = 1
        if cCorrupted.value :
            fCorrupted = 1

        if cAvailable.value == 0 :
            continue

        if cSamples + cAvailable.value > nSamples :
            cAvailable = c_int(nSamples-cSamples)

        dwf.FDwfAnalogInStatusData(hdwf, c_int(canal), byref(rgdSamples, sizeof(c_double)*cSamples), cAvailable) # get channel 1 data
        cSamples += cAvailable.value

    dwf.FDwfAnalogOutReset(hdwf, c_int(canal))
    dwf.FDwfDeviceCloseAll()

    print("Recording done")
    if fLost:
        print("Samples were lost! Reduce frequency")
    if fCorrupted:
        print("Samples could be corrupted! Reduce frequency")
    
    muestras = np.fromiter(rgdSamples, dtype = np.float)    
    return muestras

In [ ]:
# EXPERIMENTAR
# conectar el primer generador de la AD2 al primer canal del osciloscopio de la AD2 y adquirir

cantidad_de_muestras = 50000
generar_sin = True

muestras = adquirir_un_canal(cantidad_de_muestras, generar_sinusoide=generar_sin)
print(muestras.shape)

# guardar a archivo
np.savetxt('ADQUISICION_UN_CANAL.csv', muestras )

#graficar
plt.figure()
plt.plot(muestras,'b.-')
plt.show()

### 3.2 Modificar la función de la parte anterior para poder adquirir de dos canales

Modificar la función de la parte 3.1 para adquirir de los dos canales del osciloscopio.


In [ ]:
def adquirir_dos_canales(cantidad_de_muestras, frecuencia_de_muestreo=25000, 
                         rango_canal_0=10, rango_canal_1=10,
                         generar_sinusoide=False):
    '''
    Adquiere las señales conectadas a los canales ch0 y ch1 del osciloscopio.
    
    Opcionalmente genera una sinusoide por el primer canal del generador de señales por 
    si se quiere enviar esta señal al osciloscopio.
    
    Devuelve las muestras adquiridas como dos arrays de numpy.
    '''
    
    # COMPLETAR
    muestras_ch0 = adquirir_un_canal(cantidad_de_muestras=cantidad_de_muestras, frecuencia_de_muestreo=frecuencia_de_muestreo, 
                     rango_canal=rango_canal_0,
                     canal=0,
                     generar_sinusoide=generar_sinusoide)
    
    muestras_ch1 =  adquirir_un_canal(cantidad_de_muestras=cantidad_de_muestras, frecuencia_de_muestreo=frecuencia_de_muestreo, 
                     rango_canal=rango_canal_1,
                     canal=1,
                     generar_sinusoide=generar_sinusoide)   
    
    return muestras_ch0, muestras_ch1


In [ ]:
# EXPERIMENTAR 
# conectar el primer generador de la AD2 a los dos canales del osciloscopio de la AD2 y adquirir

cantidad_de_muestras = 50000
generar_sin = True

muestras_ch0, muestras_ch1 = adquirir_dos_canales(cantidad_de_muestras, generar_sinusoide=generar_sin)
print(muestras_ch0.shape, muestras_ch1.shape)


# juntar las muestras en un sólo array de tamaño (cantidad_de_muestras x 2)
muestras = np.vstack((muestras_ch0, muestras_ch1)).T
print(muestras.shape)

# guardar a archivo
np.savetxt('ADQUISICION_DOS_CANALES.csv', muestras )

#graficar
plt.figure()
plt.plot(muestras_ch0,'b', label='ch0')
plt.plot(muestras_ch1,'r', label='ch1')
plt.legend()
plt.show()